In [32]:
import gdown
import pandas as pd
import os
import csv
import re

In [33]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [34]:
download_csv()

In [35]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
possible_columns = [
    "服裝部位 Clothing Part",
    "上半身 衣服種類 Top Clothing Type",
    "下半身衣服種類 Bottom Clothing Type",
]

In [36]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)
for col in possible_columns:
    df2 = df2.drop(col, axis=1)

In [37]:
# get all feature
all_features = []
for index, row in df.iterrows():
    for col in possible_columns:
        str = df.loc[index, col]
        try:
            str = list(str.split(","))
            for s in str:
                s = s.rstrip()
                s = s.lstrip()
                if (
                    s == "針織衣 Sweater"
                    or s == "毛衣 Sweater"
                    or s == "針/織衣 Sweater"
                    or s == "毛衣、針/織衣 Sweater"
                ):
                    s = "毛衣、針織衣 Sweater"
                if s == "西裝褲 dress pants":
                    str.append("褲子 pants")
                    s = "西裝褲、裙 dress pants、skirt"
                all_features.append(s)
        except:
            pass
all_features = list(set(all_features))
for f in all_features:
    print(f)
print(len(all_features))

feature_df = pd.DataFrame({"name": all_features})
feature_df.to_csv("feature_list.csv")

Shirt
羽絨衣 Down coat / jacket
百褶裙 pleated skirt
網紗 Mesh
吊帶褲/裙 overalls/jumper skirt
毛衣、針織衣 Sweater
褲子 pants
牛仔材質 Denim
工裝褲/裙 cargo pants/skirt
緊身褲 leggings
牛仔褲/裙 jeans / demi skirt
洋裝 Dress
外套 Jacket
西裝 Suit
上半身 Top
運動褲/裙 sweat pants / sports skirt
休閒褲/裙 casual pants/skirt
連帽衣 Hoodie
背心 Vest
T-Shirt
運動材質 Sport Tops
下半身 Bottom
西裝褲、裙 dress pants、skirt
裙子 skirt
24


In [38]:
for index, row in df.iterrows():
    for fea in all_features:
        has = 0
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                        or s == "毛衣、針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has = 1
            except:
                pass
        df2.loc[index, fea] = has


print(df2)

                                             照片 Picture    品牌 Brand  \
0     https://drive.google.com/open?id=1Ki8gALrL6WEy...         H&M   
1     https://drive.google.com/open?id=1rsB7XhHjAww6...         H&M   
2     https://drive.google.com/open?id=1BEUPTgjmZ0uZ...         H&M   
3     https://drive.google.com/open?id=12MW0xvZVVNVo...         H&M   
4     https://drive.google.com/open?id=1rj0PNtlTPp5O...         H&M   
...                                                 ...         ...   
3241  https://drive.google.com/open?id=1QgxQuXE6-9UD...  Timberland   
3242  https://drive.google.com/open?id=1K9MRFTE-9vQX...  Timberland   
3243  https://drive.google.com/open?id=1Xu8rl1Brd5J4...  Timberland   
3244  https://drive.google.com/open?id=1gUB9zkZhJt3f...  Timberland   
3245  https://drive.google.com/open?id=1HdvwRzT047SG...  Timberland   

     衣服長度 Clothes Length  價格 Price  Shirt  羽絨衣 Down coat / jacket  \
0                短 short       499    0.0                     0.0   
1        

In [39]:
for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [40]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, path, quiet=False)

In [41]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        image_url = df.loc[index, "照片 Picture"]
        print("download ", index, " from ", image_url)
        download_image(image_url, path)